In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl
import kinematics
import util

In [ ]:
mpl.rc('font', size=14)

dosave = True
savedir = "./figures/"

In [ ]:
# FILE = "/exp/sbnd/data/users/munjung/osc/sbnd.df"
FILE = "/exp/sbnd/data/users/gputnam/gump_ep.df"


DETECTOR = "ICARUS"

In [ ]:
evtdf = pd.read_hdf(FILE, "eevt")
mcdf = pd.read_hdf(FILE, "mcnu")
hdr = pd.read_hdf(FILE, "hdr")

In [ ]:
if DETECTOR == "ICARUS":
    GOAL_POT = 3.41e20
    POTSTR = "$3.41\\times 10^{20}$ POT"
elif DETECTOR == "SBND":
    GOAL_POT = 6.6e20 / 3
    POTSTR = "$2.2\\times 10^{20}$ POT"

In [ ]:
POT = hdr.pot.sum()

evtdf["weight"] = GOAL_POT / POT
mcdf["weight"] = GOAL_POT / POT
GOAL_POT / POT

In [ ]:
def InFV(df):
    return util.InFV(df, 50, det=DETECTOR)

def InBeam(t):
    return (t > 0.) & (t < 1.800)

In [ ]:
def is_cosmic(df):
    return np.isnan(df.iscc)

def is_FV(df): 
    return (InFV(df.position))

def is_numu(df):
    return (np.abs(df.pdg) == 14)

def is_CC(df):
    return (df.iscc == 1)

def is_NC(df):
    return (df.iscc == 0)

def is_1p0pi(df):
     return (df.nmu == 1) & (df.np_50MeV == 1) & (df.npi == 0) & (df.npi0 == 0)

def is_signal(df):
    return is_numu(df) & is_CC(df) & is_1p0pi(df) & is_FV(df)

def is_outFV(df):
    return is_numu(df) & is_CC(df) & is_1p0pi(df) & np.invert(is_FV(df))

def is_othernumuCC(df):
    return is_numu(df) & is_CC(df) & np.invert(is_1p0pi(df)) & is_FV(df)

In [ ]:
v = mcdf.E
when = is_signal(mcdf)

_ = plt.hist(v[when])

In [ ]:
bins = np.linspace(0, 5, 21)

D, bins = np.histogram(mcdf.E[is_signal(mcdf)], bins=bins)
N, _ = np.histogram(evtdf.truth.E[is_signal(evtdf.truth)], bins=bins)

centers = (bins[:-1] + bins[1:]) / 2

In [ ]:
plt.plot(centers, N/D)

In [ ]:
def cut_all(cuts):
    ret = cuts[0]
    for c in cuts[1:]:
        ret = ret & c
    return ret

In [ ]:
def no_oddballs(df):
    return (df.nk == 0) & (df.nk0 == 0) & (df.nsm == 0) & (df.nsp == 0)
    
def no_pions(df):
    return (df.npi == 0) & (df.npi0 == 0)


def categories(df):
    return [
        (df.iscc == 1.) & (df.nmu == 1) & (df.np_20MeV == 1) & no_pions(df) & no_oddballs(df),
        (df.iscc == 1.) & (df.nmu == 1) & (df.np_20MeV == 0),
        (df.iscc == 1.) & (df.nmu == 1) & (df.np_20MeV > 1) & no_pions(df) & no_oddballs(df),
        (df.iscc == 1.) & (df.nmu == 1) & ~no_pions(df) & no_oddballs(df),
        (df.iscc == 0.),
        (df.iscc == 1.) & ((df.nmu != 1) | ~no_oddballs(df)),
        np.isnan(df.iscc),
    ]

labels = [
    r"$\nu_\mu$ CC $1p0\pi$",
    r"$\nu_\mu$ CC $0p0\pi$",
    r"$\nu_\mu$ CC $>1p0\pi$",
    r"$\nu_\mu$ CC $n\pi$",
    r"$\nu$ NC",
    r"$\nu$ Other",
    "Cosmic"

]

In [ ]:
mode_list = [0, 10, 1, 2, 3]
mode_labels = ['QE', 'MEC', 'RES', 'SIS/DIS', 'COH', "other"]
mode_colors = ["darkorchid", "royalblue", "forestgreen", "darkorange", "firebrick"]

def breakdown_mode(var, w, df, cuts):
    c = cut_all(cuts)

    ret = [var[(df.genie_mode == i) & c] for i in mode_list] 
    wret = [w[(df.genie_mode == i) & c] for i in mode_list] 
    return ret, wret


In [ ]:
top_labels = ["Signal",
              "Other $\\nu_\\mu$ CC",
              "$\\nu$ NC",
              "Out of FV",
              "Cosmic",
              "Other"]

top_colors = []
def breakdown_top(var, w, df, cuts):
    c = cut_all(cuts)
    ret = [var[is_signal(df) & c],
           var[is_othernumuCC(df) & c],
           var[is_NC(df) & c],
           var[is_outFV(df) & c],
           var[is_cosmic(df) & c],
           var[np.invert(is_signal(df) | is_othernumuCC(df) | is_NC(df) | is_outFV(df) | is_cosmic(df)) & c]
           ]
    wret = [w[is_signal(df) & c],
           w[is_othernumuCC(df) & c],
           w[is_NC(df) & c],
           w[is_outFV(df) & c],
           w[is_cosmic(df) & c],
           w[np.invert(is_signal(df) | is_othernumuCC(df) | is_NC(df) | is_outFV(df) | is_cosmic(df)) & c]
           ]
    return ret, wret

In [ ]:
evtdf["presel"] = True
cuts = [evtdf.presel]

In [ ]:
var = evtdf.mu.length
bins = np.linspace(0, 400, 21)

var = evtdf.mu.length
bins = np.linspace(0, 400, 21)

pvar, pw = breakdown_top(var, df.weight, df.truth, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR);
plt.title("%s ML Selection" % DETECTOR);
plt.tight_layout();
plt.legend();
plt.xlabel("Muon Length [cm]");
plt.show();

if dosave:
    plt.savefig(savedir + "%s_esel_muon_length.pdf" % DETECTOR);
    plt.savefig(savedir + "%s_esel_muon_length.png" % DETECTOR);

In [ ]:
tight_selection = np.isnan(evtdf.lead_gamma_energy) &\
                np.isnan(evtdf.lead_elec_energy) &\
                np.isnan(evtdf.lead_pion_length) &\
                np.isnan(evtdf.subl_muon_length) &\
                np.isnan(evtdf.subl_proton_length)

In [ ]:
cuts += [tight_selection]

In [ ]:
var = evtdf.mu.length
bins = np.linspace(0, 400, 21)

pvar, pw = breakdown_top(var, df.weight, df.truth, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR);
plt.title("%s ML Selection" % DETECTOR);
plt.tight_layout();
plt.legend();
plt.xlabel("Muon Length [cm]");
plt.show();

if dosave:
    plt.savefig(savedir + "%s_etightsel_muon_length.pdf" % DETECTOR);
    plt.savefig(savedir + "%s_etightsel_muon_length.png" % DETECTOR);

In [ ]:
def emagdf(df):
    return np.sqrt(df.I0**2 + df.I1**2 + df.I2**2)

In [ ]:
mu_p = emagdf(evtdf.mu.momentum)/1e3
mu_dir = evtdf.mu.momentum.divide(mu_p*1e3, axis=0)
mu_dir.columns = ["x", "y", "z"]

p_p = emagdf(evtdf.p.momentum)/1e3
p_dir = evtdf.p.momentum.divide(p_p*1e3, axis=0)
p_dir.columns = ["x", "y", "z"]


In [ ]:
DELP_TH = 0.25

In [ ]:
del_p = kinematics.transverse_kinematics(mu_p, mu_dir, p_p, p_dir)[0]

In [ ]:
var = del_p
bins = np.linspace(0, 1, 21)

pvar, pw = breakdown_top(var, df.weight, df.truth, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR);
plt.title("%s ML Selection" % DETECTOR);
plt.tight_layout();
plt.legend();
plt.xlabel("Transverse Momentum [GeV/c]");
plt.axvline(DELP_TH, color="k", linestyle="--")
plt.show();

if dosave:
    plt.savefig(savedir + "%s_etightsel_transverse_momentum.pdf" % DETECTOR);
    plt.savefig(savedir + "%s_etightsel_transverse_momentum.png" % DETECTOR);

In [ ]:
# transverse momentum cut
delp_cut = (del_p < DELP_TH)

cuts += [delp_cut]

In [ ]:
var = kinematics.neutrino_energy(mu_p, mu_dir, p_p, p_dir)
bins=np.linspace(0,3,21)

pvar, pw = breakdown_top(var, df.weight, df.truth, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR);
plt.title("%s ML Selection" % DETECTOR);
plt.tight_layout();
plt.legend();
plt.xlabel("Neutrino Energy [GeV]");
plt.show();

if dosave:
    plt.savefig(savedir + "%s_eseldpT_neutrino_energy.pdf" % DETECTOR);
    plt.savefig(savedir + "%s_eseldpT_neutrino_energy.png" % DETECTOR);


In [ ]:
cutnames = [
    "Pre-Selection",
    "Othr Obj. Cut",
    "$\\delta p_T$"
]

assert(len(cuts) == len(cutnames))

In [ ]:
bins = np.array([0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.5, 2, 2.5, 3])
centers = (bins[:-1] + bins[1:]) /2

varmc = mcdf.E
vardf = df.truth.E

whenmc = is_signal(mcdf)

Ns = []
for i in range(len(cuts)):
    whendf = is_signal(df.truth) & cut_all(cuts[:i+1])
    N,_ = np.histogram(vardf[whendf], weights=df.weight[whendf], bins=bins)
    Ns.append(N)
    
D,_ = np.histogram(varmc[whenmc], weights=mcdf.weight[whenmc], bins=bins)

In [ ]:
for l, N in zip(cutnames, Ns):
    plt.plot(centers, N/D, label=l);
    

plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3);
plt.xlabel("Neutrino Energy [GeV]");
plt.ylabel("Selection Efficiency");
plt.text(0.025, 0.875, DETECTOR + "\nML Selection", transform=plt.gca().transAxes, size=16)
plt.ylim([0, 1])
plt.show();

if dosave:
    plt.savefig(savedir + "%s_e_efficiency_nuE.pdf" % DETECTOR, bbox_inches="tight");
    plt.savefig(savedir + "%s_e_efficiency_nuE.png" % DETECTOR, bbox_inches="tight");

In [ ]:
bins = np.array([0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.5, 2, 2.5, 3])
centers = (bins[:-1] + bins[1:]) /2

varmc = mcdf.E
vardf = df.truth.E

whenmc = is_signal(mcdf)

Ns = []
for i in range(1, len(cuts)+1):
    whendf = is_signal(df.truth) & cut_all(cuts[:i] + cuts[i+1:])
    N,_ = np.histogram(vardf[whendf], weights=df.weight[whendf], bins=bins)
    Ns.append(N)
    
D,_ = np.histogram(varmc[whenmc], weights=mcdf.weight[whenmc], bins=bins)

In [ ]:
for l, N in zip(cutnames[1:] + ["All Cuts"], Ns):
    plt.plot(centers, N/D, label=l);
    

plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3);
plt.xlabel("Neutrino Energy [GeV]");
plt.ylabel("N-1 Efficiency");
plt.text(0.025, 0.875, DETECTOR + "\nML Selection", transform=plt.gca().transAxes, size=16)
plt.ylim([0, 1])
plt.show();

if dosave:
    plt.savefig(savedir + "%s_eN-1eff_nuE.pdf" % DETECTOR, bbox_inches="tight");
    plt.savefig(savedir + "%s_eN-1eff_nuE.png" % DETECTOR, bbox_inches="tight");